In [24]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import brickschema
import datetime
from relaxation_graphs.fast_relaxation_graph import *
from evaluation.similarity_score import *
import uuid
import pickle
from rdflib import URIRef, Literal, BNode
from queries.mortar import queries as q1
%matplotlib inline

In [25]:
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 32}

matplotlib.rc('font', **font)

In [26]:
file = "Data - Brick models/acad.ttl"

running_example = """SELECT * WHERE {
 ?sen   rdf:type   brick:Zone_Air_Temperature_Sensor .
 ?sp    rdf:type   brick:Zone_Air_Temperature_Setpoint .
 ?ahu   rdf:type   brick:AHU .
 ?ahu   brick:feeds ?thing .
 ?thing  brick:hasPoint ?sen .
 ?thing  brick:hasPoint ?sp .
}"""


## max_level = 3

In [27]:
max_level = 3
print_counter = 3
G = get_fast_relaxed_graph(query=running_example, max_level=max_level)
already_done_queries = []

In [28]:
g = brickschema.Graph(load_brick=True)
g.load_file(file)
g.expand(profile="owlrl")
select_statement = running_example.split("{")[0] + "{\n"

In [29]:
total_instances = get_total_num_instances(g, 'brick:Equipment') + get_total_num_instances(g, 'brick:Location') + get_total_num_instances(g, 'brick:Point')
total_instances

1920

In [30]:
original_uuid = list(G.nodes())[0]
original_query = G.nodes()[original_uuid]['query']

### case 1: all weights=1

In [31]:
scores = {}
for idx, node in G.nodes().data():    
    relaxed_query = list(node['query'])
    score = sim_score(original_query, relaxed_query, g, total_instances)
    
    level = node['level']
    if score in scores:
        scores[score].append(relaxed_query)
    else:
        scores[score] = [relaxed_query]

In [32]:
for score in sorted(list(scores.keys()), reverse = True):
    print("score: {}, number of relaxed queries with this score: {}".format(score, len(scores[score])))

score: 1, number of relaxed queries with this score: 9
score: 0.9931530184729493, number of relaxed queries with this score: 11
score: 0.9125109579037309, number of relaxed queries with this score: 1
score: 0.9056916551913571, number of relaxed queries with this score: 7
score: 0.9041936020061722, number of relaxed queries with this score: 4
score: 0.8994904011590579, number of relaxed queries with this score: 5
score: 0.8980026051163585, number of relaxed queries with this score: 1
score: 0.8252063379512443, number of relaxed queries with this score: 1
score: 0.7681412256571158, number of relaxed queries with this score: 11
score: 0.7628817768748756, number of relaxed queries with this score: 6
score: 0.695699098086111, number of relaxed queries with this score: 5
score: 0.6945483816763435, number of relaxed queries with this score: 1
score: 0.6909356592131295, number of relaxed queries with this score: 1
score: 0.6617400035965336, number of relaxed queries with this score: 1
score: 0

In [33]:
results = {}

In [34]:
max_score = 0
count = 0
queries_to_print1 = []

for score in sorted(list(scores.keys()), reverse = True):
    if score > max_score:
        max_score = score

    for i in scores[score]:
        if not str(i) in results:
            brick_query = generate_brick_query_from_node(i, select_statement=select_statement)
            res = run_brick_query(building_model=g, query=brick_query)
            results[str(i)] = res
        else:
            res = results[str(i)]
            
        if len(res) > 0:
            if not (i, score) in queries_to_print1:
                queries_to_print1.append((i, score))
            print(res)
        print(i, score)

    count = len(queries_to_print1)
    
    if count >= print_counter:
        break

for q, sc in queries_to_print1:
    print(q, sc)

[rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1336'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1386'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1360'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1361'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1353'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1381'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1366'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1394'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1333'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1371'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1328'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1354'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1352'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1375'), rdflib.term.URIRef('http://builds

#### case 2: triple 1,2,3,4 weights = 0.5, triple 5 weight = 1

In [51]:
# scores2 = {}
# for idx, node in G.nodes().data():    
#     relaxed_query = list(node['query'])
#     score2 = sim_score(original_query, relaxed_query, g, total_instances)
    
#     if relaxed_query[0][2] == 'brick:Supply_Air_Temperature_Sensor': #1
#         score2 = score2*0.5
#     if relaxed_query[1][2] == 'brick:Supply_Air_Temperature_Setpoint': #2
#         score2 = score2*0.5
#     if relaxed_query[2][2] == 'brick:AHU': #3
#         score2 = score2*0.5
#     if relaxed_query[3][1] == 'brick:feeds': #4
#         score2 = score2*0.5
    
#     if score2 in scores2:
#         scores2[score2].append(relaxed_query)
#     else:
#         scores2[score2] = [relaxed_query]

In [52]:
# for score2 in sorted(list(scores2.keys()), reverse = True):
#     print("score: {}, number of relaxed queries with this score: {}".format(score2, len(scores2[score2])))

In [53]:
# max_score2 = 0
# count = 0
# queries_to_print2 = []

# for score2 in sorted(list(scores2.keys()), reverse = True):
#     if score2 > max_score2:
#         max_score2 = score2

#     for i in scores2[score2]:
#         if not str(i) in results:
#             brick_query = generate_brick_query_from_node(i, select_statement=select_statement)
#             res = run_brick_query(building_model=g, query=brick_query)
#             results[str(i)] = res
#         else:
#             res = results[str(i)]

#         if len(res) > 0:
#             if not (i, score2) in queries_to_print2:
#                 queries_to_print2.append((i, score2))
#             print(res)
#         print(i, score2)
#     count = len(queries_to_print2)
    
#     if count >= print_counter:
#         break

# for q, sc in queries_to_print2:
#     print(q, sc)

### case 3: triple 1,2, weights = 0.1, triples 3,4 weights = 0.5, triple 5 weight = 1

In [45]:
scores3 = {}
for idx, node in G.nodes().data():    
    relaxed_query = list(node['query'])
    score3 = sim_score(original_query, relaxed_query, g, total_instances)
    
    if relaxed_query[0][2] == 'brick:Zone_Air_Temperature_Sensor': #1
        score3 = score3*0.1
    if relaxed_query[1][2] == 'brick:Zone_Air_Temperature_Setpoint': #2
        score3 = score3*0.1
    if relaxed_query[2][2] == 'brick:AHU': #3
        score3 = score3*0.5
    if relaxed_query[3][1] == 'brick:feeds': #4
        score3 = score3*0.5
    
    if score3 in scores3:
        scores3[score3].append(relaxed_query)
    else:
        scores3[score3] = [relaxed_query]

In [46]:
for score3 in sorted(list(scores3.keys()), reverse = True):
    print("score: {}, number of relaxed queries with this score: {}".format(score3, len(scores3[score3])))

score: 0.44974520057952894, number of relaxed queries with this score: 1
score: 0.34546782960656475, number of relaxed queries with this score: 1
score: 0.22487260028976447, number of relaxed queries with this score: 4
score: 0.22450065127908964, number of relaxed queries with this score: 1
score: 0.07628817768748757, number of relaxed queries with this score: 1
score: 0.0695699098086111, number of relaxed queries with this score: 1
score: 0.04965765092364747, number of relaxed queries with this score: 4
score: 0.04528458275956786, number of relaxed queries with this score: 3
score: 0.04520968010030861, number of relaxed queries with this score: 1
score: 0.03814408884374378, number of relaxed queries with this score: 5
score: 0.03478495490430555, number of relaxed queries with this score: 4
score: 0.03472741908381718, number of relaxed queries with this score: 1
score: 0.024828825461823734, number of relaxed queries with this score: 7
score: 0.022812773947593276, number of relaxed quer

In [47]:
max_score3 = 0
count = 0
queries_to_print3 = []

for score3 in sorted(list(scores3.keys()), reverse = True):
    if score3 > max_score3:
        max_score3 = score3

    for i in scores3[score3]:
        if not str(i) in results:
            brick_query = generate_brick_query_from_node(i, select_statement=select_statement)
            res = run_brick_query(building_model=g, query=brick_query)
            results[str(i)] = res
        else:
            res = results[str(i)]

        if len(res) > 0:
            if not (i, score3) in queries_to_print3:
                queries_to_print3.append((i, score3))
            print(res)
        print(i, score3)
    count = len(queries_to_print3)
    
    if count >= print_counter:
        break

for q, sc in queries_to_print3:
    print(q, sc)

[rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1363'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1324'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1327'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1386'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1394'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1360'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1386'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1371'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1361'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1333'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1371'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1384'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1333'), rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1336'), rdflib.term.URIRef('http://builds

In [48]:
results

{"[['?sen', 'rdf:type', 'brick:Zone_Air_Temperature_Sensor'], ['?sp', 'rdf:type', 'brick:Zone_Air_Temperature_Setpoint'], ['?ahu', 'rdf:type', 'brick:AHU'], ['?ahu', 'brick:feeds', '?thing'], ['?thing', 'brick:hasPoint', '?sen'], ['?thing', 'brick:hasPoint', '?sp']]": [rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1336'),
  rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1386'),
  rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1360'),
  rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1361'),
  rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1353'),
  rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1381'),
  rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1366'),
  rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1394'),
  rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1333'),
  rdflib.term.URIRef('http://buildsys.org/ontologies/ACAD#VAVRM1371'),
  rdflib.term.URIRef('

In [49]:
queries_to_print1

[([['?sen', 'rdf:type', 'brick:Zone_Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Zone_Air_Temperature_Setpoint'],
   ['?ahu', 'rdf:type', 'brick:AHU'],
   ['?ahu', 'brick:feeds', '?thing'],
   ['?thing', 'brick:hasPoint', '?sen'],
   ['?thing', 'brick:hasPoint', '?sp']],
  1),
 ([['?sen', 'rdf:type', 'brick:Zone_Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Zone_Air_Temperature_Setpoint'],
   ['?ahu', 'rdf:type', 'brick:AHU'],
   ['?ahu', 'brick:feeds', '?thing'],
   ['?thing', 'brick:hasPoint', '?sen'],
   ['?thing', 'brick:hasPoint+', '?sp']],
  1),
 ([['?sen', 'rdf:type', 'brick:Zone_Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Zone_Air_Temperature_Setpoint'],
   ['?ahu', 'rdf:type', 'brick:AHU'],
   ['?ahu', 'brick:feeds', '?thing'],
   ['?thing', 'brick:hasPoint+', '?sen'],
   ['?thing', 'brick:hasPoint', '?sp']],
  1),
 ([['?sen', 'rdf:type', 'brick:Zone_Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Zone_Air_Temperature_Setpoint'],
   ['?

In [54]:
# queries_to_print2

In [50]:
queries_to_print3

[([['?sen', 'rdf:type', 'brick:Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Air_Temperature_Setpoint'],
   ['?ahu', 'rdf:type', 'brick:AHU'],
   ['?ahu', 'brick:feeds+', '?thing'],
   ['?thing', 'brick:hasPoint', '?sen'],
   ['?thing', 'brick:hasPoint', '?sp']],
  0.44974520057952894),
 ([['?sen', 'rdf:type', 'brick:Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Air_Temperature_Setpoint'],
   ['?ahu', 'rdf:type', 'brick:HVAC_Equipment'],
   ['?ahu', 'brick:feeds', '?thing'],
   ['?thing', 'brick:hasPoint', '?sen'],
   ['?thing', 'brick:hasPoint', '?sp']],
  0.34546782960656475),
 ([['?sen', 'rdf:type', 'brick:Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Air_Temperature_Setpoint'],
   ['?ahu', 'rdf:type', 'brick:AHU'],
   ['?ahu', 'brick:feeds', '?thing'],
   ['?thing', 'brick:hasPoint', '?sen'],
   ['?thing', 'brick:hasPoint', '?sp']],
  0.22487260028976447),
 ([['?sen', 'rdf:type', 'brick:Air_Temperature_Sensor'],
   ['?sp', 'rdf:type', 'brick:Air_Temper